File used for features extraction for given url in csv format

In [37]:
import features_extractors.content_features as ctnfe
import features_extractors.url_features as urlfe
import features_extractors.external_features as trdfe
import features_extractors.feature_extractor as ftext
import sys
import pandas as pd
import torch
import numpy as np
from torch.utils.data import TensorDataset,DataLoader
from sklearn.model_selection import train_test_split

In [38]:
header = [
        "url", "length_url", "length_hostname", "ip", "nb_dots", "nb_hyphens", "nb_at", "nb_qm", "nb_and", "nb_or", "nb_eq", "nb_underscore",
        "nb_tilde", "nb_percent", "nb_slash", "nb_star", "nb_colon", "nb_comma", "nb_semicolumn", "nb_dollar", "nb_space", "nb_www", "nb_com", "nb_dslash",
        "http_in_path", "https_token", "ratio_digits_url", "ratio_digits_host", "punycode", "port", "tld_in_path", "tld_in_subdomain",
        "abnormal_subdomain", "nb_subdomains", "prefix_suffix", "shortening_service", "path_extension", "nb_redirection",
        "nb_external_redirection", "length_words_raw", "char_repeat", "shortest_words_raw", "shortest_word_host", "shortest_word_path",
        "longest_words_raw", "longest_word_host", "longest_word_path", "avg_words_raw", "avg_word_host", "avg_word_path", "phish_hints",
        "domain_in_brand", "brand_in_subdomain", "brand_in_path", "suspecious_tld", "statistical_report", "nb_hyperlinks", "ratio_intHyperlinks",
        "ratio_extHyperlinks", "ratio_nullHyperlinks", "nb_extCSS", "ratio_intRedirection", "ratio_extRedirection", "ratio_intErrors",
        "ratio_extErrors", "login_form", "external_favicon", "links_in_tags", "submit_email", "ratio_intMedia", "ratio_extMedia", "sfh", "iframe",
        "popup_window", "safe_anchor", "onmouseover", "right_clic", "empty_title", "domain_in_title", "domain_with_copyright", "whois_registered_domain",
        "domain_registration_length", "domain_age", "web_traffic", "dns_record", "google_index", "page_rank", "status"
]

In [39]:
# Setup input

url = input("Give url:")
# url = https://www.x-kom.pl/
print("Your url is: ", url)

Your url is:  https://www.youtube.com/


In [40]:
status = 1
features = ftext.extract_features(url, status)
print(features)

c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'input.payapi.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['https://www.youtube.com/', 24, 15, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 3, 3, 0, 7, 7, 0, 5.0, 5.0, 0, 0, 1, 0, 0, 0, 0, 18, 0.8333333333333334, 0.16666666666666666, 0.0, 0, 0.0, 0.6666666666666666, 0.0, 0.0, 0, 1, 80.0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 1, -1, 6590, 0, 0, 1, -1, 1]


In [41]:
savedModelName = 'phishing_v1.pt'
model = torch.load(savedModelName)
model.eval()

Sequential(
  (0): BatchNorm1d(86, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=86, out_features=300, bias=True)
  (2): ReLU()
  (3): Linear(in_features=300, out_features=300, bias=True)
  (4): ReLU()
  (5): Linear(in_features=300, out_features=1, bias=True)
  (6): Sigmoid()
)

In [42]:
df = pd.DataFrame(data=[features], columns=header).drop("url", axis="columns")
df.head(1)

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,24,15,0,2,0,0,0,0,0,0,...,0,0,1,-1,6590,0,0,1,-1,1


In [43]:
features = df.iloc[:, :-1]
is_phishing = df.iloc[:,-1:]

test_x_t=torch.from_numpy(features.values).float()
test_y_t=torch.from_numpy(is_phishing['status'].values).float().unsqueeze(1)

testingSet=TensorDataset(test_x_t,test_y_t)
testLoader = DataLoader(testingSet, batch_size=1, shuffle=True)

In [44]:
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        outputs = model(inputs)
        predicted = torch.round(outputs.data)
        print(predicted)

tensor([[0.]])


In [45]:
if predicted == 1:
    print('The link might be phishing!')
else:
    print('The link is not phishing!')

The link is not phishing!
